In [2]:
from tira.third_party_integrations import ensure_pyterrier_is_loaded, normalize_run
ensure_pyterrier_is_loaded()

import pyterrier as pt
import pandas as pd

In [17]:


def report_effectiveness(dataset):
    queries = pt.io.read_topics(f'../{dataset}/queries.xml', 'trecxml')
    
    #run = pd.read_json('tot-train-rerank-orig-with-scores.jsonl.gz', lines=True)
    #run['qid'] = run['qid'].apply(lambda i: str(i).split('_')[0])
    run = pd.read_json('tot-train-rerank-1-with-scores.jsonl.gz', lines=True, dtype={'qid': str, 'docno': str})
    run['qid'] = run['qid'].apply(lambda i: str(i).split('_')[0])
    run = pt.Transformer.from_df(run)

    ret = pt.Experiment([run], queries, pt.io.read_qrels(f'../{dataset}/qrels.txt'), eval_metrics=["ndcg_cut_10", 'recip_rank', 'P_10', 'recall_100', 'recall_1000', 'recall_5000'])
    
    return {i: ret.iloc[0][i] for i in ['ndcg_cut_10', 'recip_rank', 'P_10', 'recall_100', 'recall_1000', 'recall_5000']}

def report_pipeline():
    ret = {}
    for dataset in ['tot-train']:#['tot-train', 'tot-dev']:
        for k,v in report_effectiveness(dataset).items():
            ret[dataset.split('-')[1] + '_' + k] = v
    return pd.DataFrame([ret])

report_pipeline()

,train_ndcg_cut_10,train_recip_rank,train_P_10,train_recall_100,train_recall_1000,train_recall_5000
0,0.119979,0.104267,0.019333,0.413333,0.626667,0.626667


In [13]:
pd.read_json('tot-train-rerank-1-with-scores.jsonl.gz', lines=True)

,qid,docno,score,rank
0,1091,67202144,-4.019356,54
1,1091,2247831,-2.045917,10
2,1091,13239685,-4.600060,97
3,1091,3979809,-5.067544,131
4,1091,2177906,-5.323334,161
...,...,...,...,...
149995,10931,185564,-12.026022,799
149996,10931,14248757,-11.183744,330
149997,10931,57820056,-11.078108,286
149998,10931,52254757,-10.723175,196


In [16]:
pd.read_json('tot-train-rerank-orig-with-scores.jsonl.gz', lines=True)

,qid,docno,score,rank
0,109_orig,67202144,-0.607404,258
1,109_orig,2247831,-0.366710,131
2,109_orig,13239685,-0.443286,161
3,109_orig,3979809,-1.086059,536
4,109_orig,2177906,-1.665459,773
...,...,...,...,...
149995,1093_orig,185564,-0.270764,605
149996,1093_orig,14248757,-0.178572,392
149997,1093_orig,57820056,-0.133381,207
149998,1093_orig,52254757,-0.308088,655
